# ADA Project Milestone 2 - Wikidata dataset newspaper generation

## Setup and Remote dataset loading

We first import necessary libraries into the notebook

In [2]:
import numpy as np
import pandas as pd

import json
import gzip
from tqdm import tqdm

import pywikibot as pw

In [ ]:
%pip install pandas==1.3.0
%pip install tld

We mount the EPFL google drive and define access paths for the different datasets



In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [3]:
BASE_PATH = "../data/mnt/ada/"
#BASE_PATH = "/content/drive/Shareddrives/Improvise ADApt Overcome/Datasets/"

SPEAKER_PATH = BASE_PATH + "speakers/"
QUOTEBANK_PATH = BASE_PATH + "quotebank/"
WIKI_PATH = BASE_PATH + "wikipedia/"
NEWS_PATH = BASE_PATH + "newspapers/"

SPEAKER_ATTRS = SPEAKER_PATH + "speaker_attributes.parquet"
WIKIPEDIA_ATTRS = SPEAKER_PATH + "wikidata_labels_descriptions.csv.bz2"
QB_WIKIPEDIA_ATTRS = SPEAKER_PATH + "wikidata_labels_descriptions_quotebank.csv.bz2"
FULL_WIKIDUMP = WIKI_PATH + "wikidata-20211004-all.json.gz"
WIKI_URLS = WIKI_PATH + "wikiurls.json"

CLEAN_WIKI_URLS = WIKI_PATH + "clean_urls.json"
CLEAN_QUOTES = QUOTEBANK_PATH + "clean_quotes.csv.bz2"
WIKI_QUOTES = NEWS_PATH + "clean_wiki_quotes.csv.bz2"
JOURNAL_WIKIDATA = WIKI_PATH + "journal_wikidump.json"
JOURNAL_PROPS = WIKI_PATH + "journal_props.json"
JOURNAL_ATTRS = NEWS_PATH + "journal_attributes.json"



To classify speakers, we will use the `speaker_attributes` parquet file as is without further modification. Its size is small enough to be managable in RAM.

## Newspaper URL extraction

We need to generate a dataset linking news agency urls with their respective wikidata entry ids. This will allow us in the future to find patterns in the groups of news outlets by having more data about them. Once the wikidata id is obtained, it will relatively easy to obtain more information about those media outlets.

Such a table is much reduced in size compared to a full wikipedia dump, since entries are restricted to news organizations. This means the dataset can be easily used in RAM, indexed by url in pandas, as a lookup table for publisher identifiers. This is a parallelizable task, so quotebank can be split into managable chunks to add a link between quote id and publisher for each quote. 

In [ ]:
import sys
from tqdm import tqdm
from urllib.parse import urlparse
from tld import get_fld

website_prop = "P856"

def has_website(s):
  return len(s.get("claims", {}).get(website_prop, [])) > 0

def extract_urls(s):
  """Takes a JSON wikidata entry and returns the list of official websites 
  linkedto that entry. The websises are returned as unique first-level-domains: 
  'https://test.google.com/exam/pl/e' becomes 'google.com'. Returns an empty 
  list if no urls"""

    # Contains an official website?
    if len(s.get("claims", {}).get(website_prop, [])) > 0:
        urls = []
        flds = []

    # For each website, add fld to urls array
    for v in s["claims"][website_prop]:
        if (v["mainsnak"].get("datavalue", {}).get("value", None) is not None):
            url = urlparse(v["mainsnak"]["datavalue"]["value"]).netloc

            fld = get_fld(v["mainsnak"]["datavalue"]["value"], fail_silently=True)

            if fld and fld not in flds:
                flds.append(fld)

            if url not in urls:
                urls.append(url)

            return urls, flds
        else:
            return [], []


def extract_newspaper_urls(inputf, outputf):
    """Takes an input wikidata dump file and writes a list of newspaper website
    URLs. It filters entries based on if they are media companies. 
    The output file is a list of json objects, where each line is a json object. 
    Each object in the output file references a news agency, with 
    - "id" wikidata identifier of the news agency
    - "label" wikidata label of the news agency
    - "websites" list of urls related to that news agency"""
    # Do not enforce encoding here since the input encoding is correct
    with open(outputf, "w") as output_file:
        with gzip.open(inputf, 'rb') as s_file:
            for instance in s_file:
                instance = instance.decode('utf-8')
                instance = instance[:-2]
                if len(instance)==0:
                    continue

                try:
                    s = json.loads(instance.strip("\n"))
                except:
                    continue

                if s.get("labels", {}).get("en") is not None:
                    s["label"] = s["labels"]["en"]["value"]

                if s.get("labels") is not None:
                    del s["labels"]
                else:
                    continue

                # Only take wiki entries with a website into consideration
                if not has_website(s):
                    continue

                # Extract Official website 
                s["websites"], s["flds"] = extract_urls(s)

                # Remove leftovers and unnecessary attributes
                if s.get("aliases") is not None:
                    del s["aliases"]
                if s.get("descriptions") is not None:
                    del s["descriptions"]
                if s.get("sitelinks") is not None:
                    del s["sitelinks"]
                if s.get("claims") is not None:
                    del s["claims"]
                if s.get("lastrevid") is not None:
                    del s["lastrevid"]
                if s.get("type") is not None:
                    del s["type"]

                output_file.write(json.dumps(s, ensure_ascii=False) + "\n")

extract_newspaper_urls(FULL_WIKIDUMP, WIKI_URLS)


Go through every entry in the Wiki URL data to flatten `{id: ..., label: ..., flds: [...]}` into a list of `{id: ..., label: ..., flds: ...}, ...`. This is done after the fact due to the extremely long running time of the previous operation

In [ ]:
with open(WIKI_URLS) as url_file, open(CLEAN_WIKI_URLS, 'w') as clean_urls:
    for line in url_file:
        urlDict = json.loads(line)
        
        for url in urlDict["flds"]:
            new_url = {
                "id": urlDict["id"],
                "url": url
            }
            
            if "label" in urlDict:
                new_url["label"] = urlDict["label"]
            
            clean_urls.write(json.dumps(new_url, ensure_ascii=False) + "\n")

We load the flattened URL mapping and index by URL. 

In [ ]:
wiki_urls = pd.read_json(CLEAN_WIKI_URLS, lines=True)
wiki_urls.head()

In [ ]:
wiki_urls.set_index('url', inplace=True)
wiki_urls.head()

## Merging quotes and URLs

We now need to add a wikidata id columns to the quotes dataset. This is done by filtering the wiki urls
to keep only the URLS that appear in the quote dataset. We then left outer join the quotes and the wiki urls on
the URL column.

In [ ]:
quotes = pd.read_csv(CLEAN_QUOTES)
quotes.head()

Keep only the wikiurl entries appearing in the quotes dataset

In [ ]:
journal_urls = pd.DataFrame(quotes.journal.unique(), columns=['url'])

journals = journal_urls.merge(wiki_urls, left_on='url', right_index=True, how='left')
journals.head()

In [ ]:
#journals.drop_duplicates(subset=['url'])
#journals.head()
journals.query('url == "foxnews.com"').head()

In [ ]:
journals.rename(columns={'url': 'journal', 'id': 'journal_id', 'label': 'journal_label'}, inplace=True)
journals.set_index('journal', inplace=True)
journals.head()

Now that we have a URL -> QID mapping, we will query wikipedia to extract hopefully meaningful features into the dataset. We do notice that the same URL can direct to several wikipedia entries. We will take care of this in a later part of the notebook

## Enhance quote dataset with source journal

In [ ]:
wiki_quotes = quotes.merge(journals, left_on='journal', how='left', right_index=True)
wiki_quotes.sample(5)

In [ ]:
pct_wiki = 1 - wiki_quotes.journal_id.isna().sum() / len(wiki_quotes.journal_id)
print(f"{pct_wiki * 100:.2f}% of all quotes have a wikipedia editor identified")

In [ ]:
wiki_quotes.to_csv(WIKI_QUOTES, compression='bz2')

## Generate journal enhanced dataset

In [ ]:
from wiki_helpers import download_wiki_entry, get_item_claims_from_wiki

def write_wikidump(file, journal, desc, claims):    
    output = {}
    
    output["journal"] = journal.journal
    output["journal_ids"] = journal.journal_id
    output["journal_label"] = journal.journal_label
    output["description"] = desc
    output["claims"] = claims
    
    file.write(json.dumps(output, ensure_ascii=False) + "\n")

with open(JOURNAL_WIKIDATA, 'w') as fjournals:        
    for _, journal in tqdm(journals.reset_index().iterrows(), total=len(journals)):
        desc, claims = download_wiki_entry(journal.journal_id)
        write_wikidump(fjournals, journal, desc, claims)

In [ ]:
def year_extractor(raw_prop_val):    
    if not (raw_prop_val and 'time' in raw_prop_val):
        return np.nan
    
    try:
        raw_year = raw_prop_val['time'].split('-')[0]
        return int(raw_year.lstrip('+').lstrip('0'))
    except:
        return np.nan

In [ ]:
from wiki_helpers import Property

properties = [
    Property("P31","instance of"),
    Property("P361", "part of"),
    Property("P136", "genre"),
    Property("P449", "original broadcaster"),
    Property("P17", "country"),
    Property("P495", "country of origin"),
    Property("P127", "owned by"),
    Property("P159", "headquaters location"),
    Property("P571", "inception_year", extractor=year_extractor),
    Property("P3912", "newspaper format"),
    Property("P407", "language of work"),
    Property("P452", "industry"),
    Property("P1128", "employees"),
    Property("P123", "publisher"),
    Property("P131", "is located in"),
    Property("P101", "field of work"),
    Property("P641", "sport"),
]

In [ ]:
from collections import defaultdict

def extract_props_from_claims(claims, props):
    extracted = {prop.label: [] for prop in props}
        
    for prop in props:
        prop_value = prop.find_in_claims(claims)
        for val in prop_value:
            if val not in extracted[prop.label]:
                extracted[prop.label].append(val)
    
    return extracted

In [ ]:
with open(JOURNAL_PROPS, 'w') as fjournals, pd.read_json(JOURNAL_WIKIDATA, lines=True, chunksize=500) as wikidump:        
    for chunk in tqdm(wikidump, total=int(41115/500)):
        for _, row in chunk.iterrows():
            props = extract_props_from_claims(row.claims, properties)
            props["journal"] = row.journal
            props["journal_id"] = row.journal_ids
            props["journal_label"] = row.journal_label
            props["description"] = row.description
            
            fjournals.write(json.dumps(props, ensure_ascii=False) + "\n")

In [9]:
def combine_columns(group, str_cols=['description', 'journal_label', 'journal_id'], index_col='journal'):    
    
    list_columns = [col for col in group.columns if col not in str_cols and col != index_col]
    journal = {column: set() for column in list_columns}
        
    for _, row in group.iterrows():
        for column in list_columns:
            if column not in str_cols:
                res = row[column]
                
                if res:
                    journal[column] = journal[column] | set(res)
    
    journal = {key: list(val) for key, val in journal.items()}
    
    for column in str_cols:
        cat = group[column].str.cat(sep='|')
        journal[column] = cat.split('|')
        
    journal[index_col] = group[index_col].iloc[0]    
    
    return journal


In [10]:
journal_attrs = pd.read_json(JOURNAL_PROPS, lines=True)
journal_attrs.head()

,instance of,part of,genre,original broadcaster,country,country of origin,owned by,headquaters location,inception_year,newspaper format,...,industry,employees,publisher,is located in,field of work,sport,journal,journal_id,journal_label,description
0,[Q1002697],[],[],[],[Q30],[Q30],[],[],[1974],[],...,[],[],[Q1411739],[],[],[],people.com,Q33659,People,weekly American magazine
1,[Q41298],[],[],[],[],[Q30],[],[],[1977],[],...,[],[],[Q519143],[],[],[],usmagazine.com,Q549578,Us Weekly,American magazine
2,[Q5398426],[],[],[Q186068],[],[Q30],[],[],[],[],...,[],[],[],[],[],[],foxnews.com,Q6317205,Justice with Judge Jeanine,television series
3,[Q15416],[],[],[Q186068],[],[Q30],[],[],[],[],...,[],[],[],[],[],[],foxnews.com,Q7304120,Red Eye w/Greg Gutfeld,US television program
4,[Q5398426],[],[],[Q186068],[],[Q30],[],[],[],[],...,[],[],[],[],[],[],foxnews.com,Q17027753,The Real Story,television series


In [11]:
grouped = journal_attrs.groupby('journal').apply(combine_columns)
grouped = pd.DataFrame(grouped.to_list()).set_index('journal')

grouped.sample(10)

,instance of,part of,genre,original broadcaster,country,country of origin,owned by,headquaters location,inception_year,newspaper format,language of work,industry,employees,publisher,is located in,field of work,sport,description,journal_label,journal_id
journal,,,,,,,,,,,,,,,,,,,,
securitymiddleeast.com,[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]
newstribune.com,[Q1002697],[],[],[],[],[],[Q7948868],[],[],[Q665319],[],[],[],[],[],[],[],[],[Jefferson City News Tribune],[Q20710733]
thetelegram.com,[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]
usgamer.net,[Q72398691],[],[],[],[],[],[],[],[2013],[],[Q1860],[],[],[],[],[],[],[defunct video game news website],[USgamer],[Q73939073]
gizbot.com,[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]
outsidethebeltway.com,[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]
vestaviavoice.com,[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]
fox21online.com,[Q1616075],[],[],[],[Q30],[],[],[],[1994],[],[],[],[],[],[],[],[],"[Fox television affiliate in Duluth, Minnesota...",[KQDS-TV],[Q6336159]
boereport.com,[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]


In [12]:
grouped.to_json(JOURNAL_ATTRS)

In [13]:
pd.read_json(JOURNAL_ATTRS)

,instance of,part of,genre,original broadcaster,country,country of origin,owned by,headquaters location,inception_year,newspaper format,language of work,industry,employees,publisher,is located in,field of work,sport,description,journal_label,journal_id
1011now.com,[Q1616075],[],[],[],[Q30],[],[],[Q28260],[1953],[],[],[],[],[],[],[],[],"[CBS television affiliate in Lincoln, Nebraska...",[KOLN],[Q6334998]
1070thefan.com,[Q14350],[],[],[],[Q30],[],[Q5373589],[],[2007],[],[],[],[],[],[Q1415],[],[],[radio station from Indianapolis],[WFNI],[Q7949418]
107jamz.com,[Q14350],[],[],[],[Q30],[],[Q7830269],[],[],[],[],[],[],[],[Q1588],[],[],[Urban contemporary radio station in Lake Arth...,[KJMH],[Q6331512]
10news.com,[Q1616075],[],[],[],[Q30],[],[Q283457],[Q16552],"[1953, 1997]",[],[],[],[],[],[],[],[],"[channel 10 television station in San Diego, C...","[KGTV, KZSD-LP]","[Q3191396, Q16931724]"
1130thetiger.com,[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zalebs.com,[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]
zawya.com,[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]
zdnet.com,"[Q35127, Q4830453]",[],[],[],[Q30],[],[Q7305191],[],[1991],[],[],[],[],[],[],[],[],[business technology news website],[ZDNet],[Q2457578]
zeibiz.com,[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]
